### Header

In [12]:
# import libraries

# maths
import numpy as np
import pandas as pd
import scipy.stats as stats
from pandas.api.types import is_numeric_dtype

# visual
#from matplotlib_venn import venn2
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# modelling
from sklearn import linear_model
from sklearn.linear_model import Ridge,LinearRegression,LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error,confusion_matrix,accuracy_score
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.utils import resample

# nlp
from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy
from spacy.tokens import Doc

# web
import requests
import json

# others
import os
import re
import time
import datetime as datetime

In [3]:
# file paths

input_path = '../data/2_input/'
clean_path = '../data/3_clean/'
output_path = '../data/4_output/'

image_path = '../images/'

### Functions

### Import Data

In [5]:
# import clean data

df_train = pd.read_csv(clean_path + 'train_clean.csv')
df_test = pd.read_csv(clean_path + 'test_clean.csv')
#df_weather = pd.read_csv(clean_path + 'weather_clean.csv')
#df_spray = pd.read_csv(clean_path + 'spray_clean.csv')

In [10]:
majority_class = df_train[df_train['wnvpresent']==0]
minority_class = df_train[df_train['wnvpresent']==1]

### Preprocessing

In [15]:
minority_upsampled = resample( minority_class, replace=True, n_samples=majority_class.shape[0], random_state=42)
train_resampled = pd.concat([minority_upsampled,majority_class])
train_resampled.wnvpresent.value_counts()

1    9955
0    9955
Name: wnvpresent, dtype: int64

In [16]:
train_resampled.head()

,Unnamed: 0,species,block,street,trap,nummosquitos,wnvpresent,year,month,day,latitude,longitude
2181,2181,CULEX PIPIENS,38,E 115TH ST,T215,30,1,2007,8,17,41,-87
9852,9852,CULEX PIPIENS,82,S KOSTNER AVE,T225,21,1,2013,8,22,41,-87
7198,7198,CULEX RESTUANS,10,W OHARE,T903,7,1,2011,8,5,41,-87
2192,2192,CULEX PIPIENS,40,E 130TH ST,T221,50,1,2007,8,17,41,-87
1900,1900,CULEX PIPIENS,46,N MILWAUKEE AVE,T003,49,1,2007,8,15,41,-87
